In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import pydicom as dicom
import scipy.misc
from glob import glob
import os

#PLEASE RUN THIS CELL 
import requests
import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom

from skimage.io import imread
from scipy.misc import imresize
from sklearn.model_selection import train_test_split


from keras.models import Sequential, Model, load_model
from keras.layers import *
from keras.utils import np_utils, to_categorical
from keras.losses import binary_crossentropy
from keras import backend as K,objectives
from keras.losses import mse, binary_crossentropy
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop
from keras.initializers import RandomNormal
from keras.preprocessing import image

import random


from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

Using TensorFlow backend.


In [10]:
import cv2

In [2]:
mass_train_attrs = pd.read_csv('attributes/mass_case_description_train_set.csv')
mass_test_attrs = pd.read_csv('attributes/mass_case_description_test_set.csv')

In [3]:
datasets = ["mass_training",
            "mass_testing"]

In [16]:
for dataset in datasets:
    im_strs = glob('raw_data/' + dataset + '_data/CBIS-DDSM/*/*/*/*')
    attrs = []
    ids = []

    for i in range(len(im_strs)):
        try:
            im_pxl = dicom.dcmread(im_strs[i]).pixel_array
        except ValueError:
            print('Error: ')
            print(dataset + ' - ' + str(i))
        scipy.misc.imsave('images/' + dataset + '/' + str(i) + '.jpg', im_pxl)

        attrs.append(im_strs[i].split('/')[3])
        ids.append(str(i) + '.jpg') 
    
    attribute_frame = pd.DataFrame()
    attribute_frame['attribute'] = attrs
    attribute_frame['ids'] = ids
    attribute_frame.to_csv('attributes/' + dataset + '_labels.csv')
    

In [34]:
mass_train_attrs['ROI mask file path'][92].split('/')[1] == mass_train_attrs['cropped image file path'][92].split('/')[1]

True

In [33]:
mass_train_attrs['ROI mask file path'][93].split('/')[1] == mass_train_attrs['cropped image file path'][93].split('/')[1]

False

In [35]:
mass_train_attrs.head()

,patient_id,breast_density,left or right breast,image view,abnormality id,abnormality type,mass shape,mass margins,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00001,3,LEFT,CC,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00001_LEFT_CC_1/1.3.6.1.4.1.95...
1,P_00001,3,LEFT,MLO,1,mass,IRREGULAR-ARCHITECTURAL_DISTORTION,SPICULATED,4,MALIGNANT,4,Mass-Training_P_00001_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00001_LEFT_MLO_1/1.3.6.1.4.1.9...
2,P_00004,3,LEFT,CC,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_CC/1.3.6.1.4.1.9590...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...,Mass-Training_P_00004_LEFT_CC_1/1.3.6.1.4.1.95...
3,P_00004,3,LEFT,MLO,1,mass,ARCHITECTURAL_DISTORTION,ILL_DEFINED,4,BENIGN,3,Mass-Training_P_00004_LEFT_MLO/1.3.6.1.4.1.959...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...,Mass-Training_P_00004_LEFT_MLO_1/1.3.6.1.4.1.9...
4,P_00004,3,RIGHT,MLO,1,mass,OVAL,CIRCUMSCRIBED,4,BENIGN,5,Mass-Training_P_00004_RIGHT_MLO/1.3.6.1.4.1.95...,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....,Mass-Training_P_00004_RIGHT_MLO_1/1.3.6.1.4.1....


In [11]:
DATA_FOLDER = 'raw_data'
ATTR_FOLDER = 'attributes'

datasets = ['mass']
subsets = ['train', 'test']

OUTPUT_FOLDER = 'clean_data'
IM_OUTPUT = 'images'
ROI_OUTPUT = 'masks'
CROP_OUTPUT = 'crops'

IM_DIMS = (512, 512)

In [13]:
for dataset in datasets:
    for subset in subsets:
        df_attrs = pd.read_csv(f'{ATTR_FOLDER}/{dataset}_case_description_{subset}_set.csv')

        for i in range(len(df_attrs)):
            # get paths
            im_path_lst = df_attrs['image file path'][i].split('/')
            roi_path_lst = df_attrs['ROI mask file path'][i].split('/')
            crop_path_lst = df_attrs['cropped image file path'][i].split('/')

            im_path = glob(
                f'{DATA_FOLDER}/{dataset}_{subset}ing_data/CBIS-DDSM/{im_path_lst[0]}/*/*/*'
            )
    
            crop_paths = glob(
                f'{DATA_FOLDER}/{dataset}_{subset}ing_ROI_data/CBIS-DDSM/{crop_path_lst[0]}/*/*/*'
            )

            # if images found for both types save to new folders
            if im_path and crop_paths:
                
                # original image
                try:
                    im_pxl = dicom.dcmread(im_path[0]).pixel_array
                except ValueError:
                    print('Error: ')
                    print(im_path[0])
                
                im_pxl = cv2.resize(im_pxl, IM_DIMS)
                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{IM_OUTPUT}/{i}.tif', im_pxl)

                # cropped and masks
                crop_path1, crop_path2 = crop_paths
                try: 
                    crop_pxl_1 = dicom.dcmread(crop_path1).pixel_array
                    crop_pxl_2 = dicom.dcmread(crop_path2).pixel_array
                except ValueError:
                    print('Error: ')
                    print(crop_path1)

                # set temporary values
                bigger = crop_pxl_1
                smaller = crop_pxl_2
                if os.path.getsize(crop_path1) < os.path.getsize(crop_path2):
                    bigger = crop_pxl_2
                    smaller = crop_pxl_1

                smaller = cv2.resize(smaller, IM_DIMS)
                bigger = cv2.resize(bigger, IM_DIMS)

                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{CROP_OUTPUT}/{i}.tif', smaller)
                scipy.misc.imsave(f'{OUTPUT_FOLDER}/{dataset}/{subset}/{ROI_OUTPUT}/{i}.tif', bigger)


In [ ]:
for i in range(mass_train_attrs.shape[0]):
    

In [ ]:
datasets_ROI = ["mass_training_ROI", "mass_testing_ROI"]

In [ ]:
for dataset in datasets_ROI:
    im_strs = glob('raw_data/' + dataset + '_data/CBIS-DDSM/*/')
    attrs = []
    ids = []

    for i in range(len(im_strs)):
        if "Mass-Training_P_00328_RIGHT_CC" in im_strs[i].split('/')[3]:
            print(im_strs[i].split('/')[3])
        #print(im_strs[i].split('/')[3])
        #attrs.append(im_strs[i].split('/')[3])
        #ids.append(str(i) + '.jpg')
    
    

In [ ]:
pic1 = glob(im_strs[i] + '/*/*/*')[0]
        pic2 = glob(im_strs[i] + '/*/*/*')[1]
        try:
            im_pxl_1 = dicom.dcmread(pic1).pixel_array
            im_pxl_2 = dicom.dcmread(pic2).pixel_array
        except ValueError:
            print('Error: ')
            print(dataset + ' - ' + str(i))
        
        if os.path.getsize(pic1) > os.path.getsize(pic2):
            scipy.misc.imsave('ROI_images/' + dataset + '/' + str(i) + '.jpg', im_pxl)
        else:
            scipy.misc.imsave('ROI_images/' + dataset + '/' + str(i) + '.jpg', im_px2)

            
            
            attribute_frame = pd.DataFrame()
    attribute_frame['attribute'] = attrs
    attribute_frame['ids'] = ids
    attribute_frame
    attribute_frame.to_csv(dataset + "_labels.csv")

In [ ]:
m = glob('FinalProject/' + 'mass_training_ROI' + '_data/CBIS-DDSM/*')[0]
glob(m + '/*/*/*')

In [ ]:
glob('FinalProject/' + 'mass_training_ROI' + '_data/CBIS-DDSM/Mass-Training_P_00108_LEFT_MLO_1/*/*/*')

In [ ]:
import os
os.path.getsize('FinalProject/mass_training_ROI_data/CBIS-DDSM/Mass-Training_P_00108_LEFT_MLO_1/07-21-2016-DDSM-74722/1-ROI mask images-57321/000000.dcm')

In [ ]:
os.path.getsize('FinalProject/mass_training_ROI_data/CBIS-DDSM/Mass-Training_P_00108_LEFT_MLO_1/07-20-2016-DDSM-43543/1-cropped images-71981/000000.dcm')